# Take pictures

In [6]:
import cv2


cap_left = cv2.VideoCapture(1,cv2.CAP_DSHOW)
cap_right = cv2.VideoCapture(0,cv2.CAP_DSHOW)


num = 0


while cap_left.isOpened():

    succes1, img_left = cap_left.read()
    succes2, img_right = cap_right.read()


    k = cv2.waitKey(5)

    if k == 27:
        break
    elif k == ord('s'): # wait for 's' key to save and exit
        cv2.imwrite('images/stereoLeft/imageL' + str(num) + '.png', img_left)
        cv2.imwrite('images/stereoRight/imageR' + str(num) + '.png', img_right)
        print("images saved!")
        num += 1

    cv2.imshow('Camera Left',img_left)
    cv2.imshow('Camera Right',img_right)
    
cap_left.release()
cap_right.release()
cv2.destroyAllWindows()


In [5]:
cap_left.release()
cap_right.release()
cv2.destroyAllWindows()

# Calibrate

In [16]:
import numpy as np
import cv2 as cv
import glob



################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (9,6)
frameSize = (640,480)



# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)
objp = objp*23

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
imgpointsR = [] # 2d points in image plane.


imagesLeft = glob.glob('images/stereoLeft/*.png')
imagesRight = glob.glob('images/stereoRight/*.png')

for imgLeft, imgRight in zip(imagesLeft, imagesRight):

    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Draw and display the corners
        cv.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        cv.imshow('img left', imgL)
        cv.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        cv.imshow('img right', imgR)
        cv.waitKey(1000)


cv.destroyAllWindows()




############## CALIBRATION #######################################################

retL, cameraMatrixL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, frameSize, None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))



########## Stereo Vision Calibration #############################################

flags = 0
flags |= cv.CALIB_FIX_INTRINSIC
# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
#retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)
retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR, cameraMatrixL, distL, cameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)
###### OBS: rot and trans are from camera left to camera right, Tright_left, so the final frame is the right frame


#print(newCameraMatrixL)
#print(newCameraMatrixR)


################################### SAVING ALL ###############################
np.save('cameraMatrixR.npy', cameraMatrixR)
np.save('cameraMatrixL.npy', cameraMatrixL)
np.save('rot.npy', rot)
np.save('trans.npy', trans)
np.save('distL.npy', distL)
np.save('distR.npy', distR)


################################### CALCULING THE ERROR ###############################
mean_errorR = 0
mean_errorL = 0

for i in range(len(objpoints)):
    imgpoints2R, _ = cv.projectPoints(objpoints[i], rvecsR[i], tvecsR[i], cameraMatrixR, distR)
    errorR = cv.norm(imgpointsR[i], imgpoints2R, cv.NORM_L2)/len(imgpoints2R)
    
    imgpoints2L, _ = cv.projectPoints(objpoints[i], rvecsL[i], tvecsL[i], cameraMatrixL, distL)
    errorL = cv.norm(imgpointsL[i], imgpoints2L, cv.NORM_L2)/len(imgpoints2L)
    
    mean_errorR += errorR
    mean_errorL += errorL
print( "total error RIGHT: {}".format(mean_errorR/len(objpoints)) )
print( "total error LEFT: {}".format(mean_errorL/len(objpoints)) )


total error RIGHT: 0.015010832901206125
total error LEFT: 0.014473227647724966


# Checking the calibration

In [17]:
import numpy as np

def triangulation(p_1,p_2,inv_k_1,inv_k_2,Rot,trans):
    p1 = inv_k_1@p_1
    p2 = inv_k_2@p_2
    lamb = np.linalg.pinv(np.concatenate((p1,-Rot@p2),1))@trans
    return lamb[0,0]*p1




def track_yellow(frame,colorLower,colorUpper):
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, colorLower, colorUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    radius = 0
    center = (0,0)

    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
    if radius > 10:
        cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
        cv2.circle(frame, center, 5, (0, 0, 255), -1)
    
    return center,frame




def draw_trajectory(frame,pts):
    for i in range(1, len(pts)):
        if pts[i - 1] is None or pts[i] is None or pts[i] == (0,0) or pts[i - 1] == (0,0):
            continue
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), 5)
    return frame


In [24]:
import imutils

cap_right = cv2.VideoCapture(0,cv2.CAP_DSHOW)                    
cap_left =  cv2.VideoCapture(1,cv2.CAP_DSHOW)

cap_right.set(cv2.CAP_PROP_EXPOSURE, -6)
cap_left.set(cv2.CAP_PROP_EXPOSURE, -6)


pts_left = []
pts_right = []

colorLower_right = (22, 141, 106)
colorUpper_right = (30, 255, 255)

colorLower_left = (27, 116, 69)
colorUpper_left = (35, 255, 255)



while True:
    ret, frame_right = cap_right.read()
    ret, frame_left = cap_left.read()
    
    
    #frame_left = cv2.undistort(frame_left, cameraMatrixL, distL, None, newCameraMatrixL)
    #frame_right = cv2.undistort(frame_right, cameraMatrixR, distR, None, newCameraMatrixR)
    #x, y, w, h = roi_L
    #frame_left = frame_left[y:y+h, x:x+w]
    #x, y, w, h = roi_R
    #frame_right = frame_right[y:y+h, x:x+w] 
    
    
    center_left,frame_left = track_yellow(frame_left,colorLower_left,colorUpper_left)
    center_right,frame_right = track_yellow(frame_right,colorLower_right,colorUpper_right)

    pts_left.append(center_left)
    pts_right.append(center_right)
    
    #frame_left = draw_trajectory(frame_left,pts_left)
    #frame_right = draw_trajectory(frame_right,pts_right)
    
    #xyz = triangulation(np.array([[center_right[0]],[center_right[1]],[1]]),np.array([[center_left[0]],[center_left[1]],[1]]),np.linalg.inv(newCameraMatrixR),np.linalg.inv(newCameraMatrixL),rot,trans)    
    xyz = triangulation(np.array([[center_right[0]],[center_right[1]],[1]]),np.array([[center_left[0]],[center_left[1]],[1]]),np.linalg.inv(cameraMatrixR),np.linalg.inv(cameraMatrixL),rot,trans)    
    cv2.putText(frame_right,text='X = '+str(xyz[0]),org=(10,100),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=3,color=(255,255,255),thickness=3,lineType=cv2.LINE_AA)
    cv2.putText(frame_right,text='Y = '+str(xyz[1]),org=(10,200),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=3,color=(255,255,255),thickness=3,lineType=cv2.LINE_AA)
    cv2.putText(frame_right,text='z = '+str(xyz[2]),org=(10,300),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=3,color=(255,255,255),thickness=3,lineType=cv2.LINE_AA)

        
    cv2.imshow("Frame left", frame_left)
    cv2.imshow("Frame right", frame_right)
    
    

    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break    

cap_right.release()
cap_left.release()
cv2.destroyAllWindows()

In [23]:
cap_right.release()
cap_left.release()
cv2.destroyAllWindows()

In [12]:
cameraMatrixR

array([[688.37747806,   0.        , 350.61857173],
       [  0.        , 695.37834373, 252.95590113],
       [  0.        ,   0.        ,   1.        ]])

In [7]:
cameraMatrixL

array([[691.38232726,   0.        , 343.48642014],
       [  0.        , 696.34584502, 244.08655323],
       [  0.        ,   0.        ,   1.        ]])

In [5]:
rot

array([[ 0.9455398 ,  0.08450312,  0.31434649],
       [-0.16010466,  0.96155752,  0.22310007],
       [-0.28340958, -0.26127833,  0.92272024]])

In [6]:
trans

array([[-365.14317637],
       [-287.46720512],
       [  61.04584426]])